# Notebook حول هذا 
* Riiid عبارة عن مجموعة من الأشياء التي يجب أن تعرفها قبل حل مسابقة
* حسب الحاجة (بعض التحديثات قيد الإنشاء) Notebook سأقوم  بتحديث 
# Riiid حول مسابقة   
### منظم المسابقة
* https://riiidlabs.ai/solutions riid مختبر 
* مختبريستخدم الذكاء الاصطناعي لحل المعظلات التي تستعصي على المجال الصناعي. وهو الآن يعمل على تطوير التعليم عن بعد
## الهدف
* Riiid! Answer Correctness Prediction/!إستخدام خوارزميات الذكاء الاصطناعي لمعرفة هل الطالب سيجيب على الأسئلة القادمة أو لا          
* Track knowledge states of 1M+ students in the wild  تتبع حالات المعرفة لأكثر من مليون طالب عن بعد                                     
## خلفية المسابقة
* في الآونة الحالية أكثر من 260 مليون طفل و شاب محرومون من حقهم في التعلم، بسب كوفيد لذلك يجب التفكير عن بعد
## ماذا تفعل في المنافسة
* Riiid الخاصة بـ EdNet إنطلاقا من مسار الطالب في المنصة، يجب عليك التنبؤ الدقيق بإجاباته المستقبلية على المنصة. استفد من مهارات التعلم الآلي لعمل تنبؤات باستخدام بيانات
  ثم قم بإنشاء خوارزمية لـ "تتبع تتبع الإجابات"، والتي تمثل حصيلة الطالب بمرور الوقت.
* في فبراير2021 حول تعليم  الذكاء الإصطناعي AAAI-2021ستتم دعوة الفريق الحائز على المراتب الأولى لتقديم نموذجهم في ورشة عمل

# البيانات المستخدمة في المسابقة

In [ ]:
import numpy as np #linear algebra الجبر الخطي
import pandas as pd # معالجة البيانات data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
#(/ kaggle / working /) يمكنك كتابة ما يصل إلى 20 جيجا بايت في الدليل الحالي
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# لكن لن يتم حفظها / kaggle / temp / يمكنك أيضًا كتابة ملفات مؤقتة إلى 

## train.csv (بيانات التدريب)

In [ ]:
# نظرًا لوجود الكثير من البيانات، قرأت ما يصل إلى مليون سطروأعرضها لك في الجدول أسفله 
train_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/train.csv",nrows=10**6 )
train_df

In [ ]:
# دعنا نرى عدد المتغيرات المتفردة لكل عمود
train_df.nunique()

In [ ]:
train_df['content_type_id'].value_counts().to_dict()

In [ ]:
# الحصول على عدد السجلات لكل مستخدم
# توصلت إلى أن أغلب المستخدمين يملكون أكثر من 10000 سجل
train_df['user_id'].value_counts(sort=True,ascending=False)

In [ ]:
train_df['task_container_id'].value_counts(sort=True,ascending=False)

In [ ]:
train_df['prior_question_had_explanation'].value_counts(sort=True,ascending=False).to_dict()

In [ ]:
# اقرأ لملف CSV
train_df_csv = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/train.csv",nrows=1000)
train_df_csv.to_csv("train_df_1000.csv")

## وصف أعمدة بيانات التدريب لدينا (train.csv)

### row_id(int64)
* ID code for the row/ كود التعريف للصف  
### timestamp(int64)
*  the time in milliseconds between this user interaction and the first event completion from that user/ الوقت بالمللي ثانية بين تفاعل المستخدم مع المنصة وأول حدث مكتمل.
### user_id: (int32)
* ID code for the user كود تعريف المستخدم.
### content_id: (int16)
* ID code for the user interaction كود تعريف تفاعل للمستخدم مع المنصة 
### content_type_id: (int8)
* 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture
* 0: إذا كان الحدث عبارة عن سؤال تم طرحه على المستخدم
* 1:حدث مشهادة المحاضرة من طرف المستخدم
### task_container_id: (int16) 
* Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.
* task_container_id كود تعريف لمجموعة الأسئلة أو المحاضرات. على سبيل المثال ، قد يرى المستخدم ثلاثة أسئلة متتالية قبل رؤية التفسيرات لأي منها. سيشترك هؤلاء الثلاثة جميعًا 
### user_answer: (int8)
* the user's answer to the question, if any. Read -1 as null, for lectures/ إجابة المستخدم على السؤال ، إن وجد 1- يعني لاغية للمحاضرات.
### answered_correctly: (int8)  
* if the user responded correctly. Read -1 as null, for lectures/ إذا استجاب المستخدم بشكل صحيح. قراءة 1- لاغية للمحاضرات.
### prior_question_elapsed_time: (float32)
*  The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.
* متوسط الوقت بالمللي ثانية الذي استغرقه المستخدم للإجابة على كل سؤال في مجموع الأسئلة السابقة ، مع تجاهل أي محاضرات بينهما. لاغية لمجموعة الأسئلة أو المحاضرة الأولى للمستخدم. 
* لاحظ أن الوقت هو متوسط الوقت الذي يستغرقه المستخدم لحل كل سؤال في الحزمة السابقة  
### prior_question_had_explanation: (bool)
* Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.
* سواء رأى المستخدم شرحًا أم لا والإجابة (الردود) الصحيحة بعد الإجابة على مجموع الأسئلة السابقة ، وتجاهل أي محاضرات بينهما. تتم مشاركة القيمة عبر مجموع أسئلة واحدة ، وتكون خالية لمجموعة الأسئلة أو المحاضرة الأولى للمستخدم. عادةً ما كانت الأسئلة العديدة الأولى التي يراها المستخدم جزءًا من اختبار تشخيصي حيث لم يحصلوا على أي تعليقات.
* {True: 889887, False: 106297}باختصار هل رأيت التعليق بعد إجابة السؤال في المرة السابقة؟ السؤال الأول لاغٍ لأنك لا تستطيع رؤية التعليق (بالطبع). القيم الفعلية هي 

## questions.csv (بيانات السؤال)

In [ ]:
questions_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/questions.csv")
questions_df

In [ ]:
# دعنا نرى عدد المتغيرات المتفردة لكل عمود
questions_df.nunique()

In [ ]:
questions_df['bundle_id'].value_counts(sort=True,ascending=False)

## وصف أعمدة بيانات التدريب لدينا (questions.csv)
### question_id:
* foreign key for the train/test content_id column, when the content type is question (0) 
* عندما يكون نوع المحتوى سؤالاً (0) (train / question) تعتبرالمفتاح بين     
### bundle_id:
* code for which questions are served together/الكود الذي تم توفير السؤال به
### correct_answer:
* the answer to the question. Can be compared with the train user_answer column to check if the user was right.
* train.csv في "user_answer" أجب على السؤال, يمكنك التحقق من صحة المستخدم من خلال مقارنته مع عمود 
### part:
* the relevant section of the TOEIC testTOEIC/TOEICالقسم المتعلق باختبار .
### tags:
* one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.
* واحد أو أكثر من رموز العلامات التفصيلية للسؤال. لم يتم توفير أي معنى للعلامة ، ولكن هذه الرموز كافية لوضع السؤال معًا.

## lectures.csv(البيانات الوصفية للمحاضرة)
* .البيانات الوصفية للمحاضرات التي يشاهدها المستخدمون أثناء تقدمهم في التعليم

In [ ]:
lectures_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/lectures.csv")
lectures_df

## وصف أعمدة بيانات التدريب لدينا (lectures.csv)
### lecture_id:
* foreign key for the train/test content_id column, when the content type is lecture (1).
* (1) إذا كان نوع المحتوى عبارة عن محاضرة (train / lectures)تعتبرالمفتاح بين 
### part:
* top level category code for the lecture.
* رمز فئة المستوى الأعلى للمحاضرات.
### tag:
* one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.
* .رمز علامة واحدة للمحاضرة. لن يتم توفير معنى العلامات ، ولكن هذه الرموز كافية لتجميع المحاضرات معًا.
### type_of:
* brief description of the core purpose of the lecture
* .وصف موجز للغرض الأساسي من المحاضرة

## example_test.csv(اختبار مجموعة البيانات)
train.csv التنسيق مطابق تقريبًا لتنسيق .time series API ثلاث مجموعات من عينات بيانات مجموعة الاختبار المقدمة بواسطة   .     
هناك عمودين مختلفين يعكسان المعلومات المتوفرة بالفعل لمدرسي الذكاء الاصطناعي في أي وقت ، ولكن بدلاً من إظهار معلومات مستخدم واحد بالضبط في كل مرة ، نقوم بتجميع تفاعلات المستخدم من أجل الأداء يظهر. بعض المستخدمين في مجموعات اختبار مخفية لا تظهر في بيانات التدريب ، مما يحاكي التحدي المتمثل في التكيف السريع مع نمذجة الوافدين الجدد على موقع الويب     

In [ ]:
test_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/example_test.csv")
test_df

## وصف أعمدة بيانات التدريب لدينا (example_test.csv)
### prior_group_responses (string)
* Provides all of the user_answer entries for previous group in a string representation of a list in the first row of the group. All other rows in each group are null. If you are using Python, you will likely want to call eval on the non-null rows. Some rows may be null, or empty lists.
* .للمجموعة السابقة في تمثيل سلسلة لقائمة في الصف الأول من المجموعة. جميع الصفوف الأخرى في كل مجموعة خالية user_answer يوفر جميع إدخالات            
* على غير- صفوف فارغة. قد تكون بعض الصفوف فارغة أو قوائم فارغة Eval فمن المحتمل أن ترغب في استدعاء ،Python إذا كنت تستخدم  
### prior_group_answers_correct (string)
* provides all the answered_correctly field for previous group, with the same format and caveats as prior_group_responses. Some rows may be null, or empty lists.
* يوفر جميع الحقول التي تم الرد عليها بشكل صحيح للمجموعة السابقة ، بنفس التنسيق والتحذيرات مثل استجابات المجموعات السابقة. قد تكون بعض الصفوف فارغة أو قائمة فارغة.

## example_sample_submission.csv (إرسال نموذج ملف)

In [ ]:
submission_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv")
submission_df

### تفاصيل واجهة API
*  Refer to the starter notebook for an example of how to complete a submission. The time-series API has changed somewhat from previous competitions!
* الخاص بك، وإذا صادفتك صعوبات الرجو طرحها في التعليقات و سأجيبك Notebook لمعرفة كيفية إرسال Starter Notebook المرجو مراجعة    
 

*  You should not try to submit anything for the rows that contain lectures.
*  (lectures) لا تحاول إرسال أي شيء  يحتوي على أي سطرمن بيانات

* The API provides user interactions groups in the order in which they occurred. Each group will contain interactions from many different users, but no more than one task_container_id of questions from any single user. Each group has between 1 and 1000 users.
* مجموع تفاعلا ت المستخدم بترتيب الحدوث (API) توفر واجهة 
* لكل مستخدم واحد (task_container_id)يحتوي على واحد (question) تحتوي كل مجموعة على العديد من تفاعلات المستخدمين المختلفين، ولكن ملف من الأسئلة 
* تضم كل مجموعة من 1 إلى 1000 مستخدم

* Expect to see roughly 2.5 million questions in the hidden test set.
* من المتوقع أن تعرض مجموعة الاختبار المخفية حوالي 2.5 مليون سؤال.

* The API will load up to 1 GB of the test set data in memory after initialization. The initialization step (env.iter_test()) will require meaningfully more memory than that; we recommend you do not load your model until after making that call. The API will also consume roughly 15 minutes of runtime for loading and serving the data, but will also obfuscate the true runtime for all submissions.
* (env.iter_test())  بتحميل ما يصل إلى 1 جيجابايت من بيانات مجموعة الاختبار في الذاكرة بعد الإعداد. ستتطلب تهيئة  فضاء العمل (API)ستقوم واجهة 
* ذاكرة أكثر من ذلك ؛ نوصيك بعدم تحميل النموذج الخاص بك إلا بعد إجراء تلك الإعدادات. ستستهلك واجهة  أيضًا ما يقرب من 15 دقيقة من وقت التشغيل لتحميل البيانات وتقديمها ولكنها ستخفي أيضًا وقت التشغيل الحقيقي لجميع عمليات الإرسال
* The API loads the data using the types specified above (int32 for user_id, int8 for content_type_id, etc).
*  (int32 لـ user_id ، int8 لـ content_type_id ، إلخ)    بتحميل البيانات باستخدام النوع المحدد أعلاه(API) توفر واجهة   

# (modelization)في الأيام القادمة سأقوم بإظافة تحليل البيانات و النمذجة       
# لا تبخل علي بالتعاليقات و النصائح